In [1]:
# Importando bibliotecas que serão utilizadas

import tweepy as tt
import pandas as pd

from api_keys import twitter_key, azure_key

In [2]:
# Cursores

azure = azure_key()

In [3]:
# Importando keys do Twiitter APP

api_key, api_key_secret, acess_token, acess_token_secret = twitter_key()

In [4]:
# Gerando autorização do Tweepy

auth = tt.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(acess_token, acess_token_secret)
api = tt.API(auth)

In [5]:
# Função para coletar dados do Twitter sobre um determinado tema

def twitter_analize(theme, since, lang, items):

    photos_url = []
    users_data = []

    # Coletando dados de tweets sobre temas específicos

    tweets = tt.Cursor(api.search_tweets, lang = lang, since = since, q = theme).items(items)

    for c in tweets:
        
        id = c._json['id']
        user_name = c.user.screen_name
        text = c.text

        photo_url, photo_tags, photo_tags_confidence, photo_descriptions, photo_descriptions_confidence, photo_celebrities, photo_celebrities_confidence = '', '', '', '', '', '', ''

        user_data_dict = {'id': id, 'user_name': user_name, 'tweet': text,
                       'photo_url': photo_url, 'photo_tags': photo_tags, 'photo_tags_confidence': photo_tags_confidence, 
                       'photo_descriptions': photo_descriptions, 'photo_descriptions_confidence': photo_descriptions_confidence, 
                       'photo_celebrities': photo_celebrities, 'photo_celebrities_confidence': photo_celebrities_confidence}
                       
        if 'media' in c.entities:
            url = c.entities['media'][0]['media_url']
            
            if url not in photos_url:
                photos_url.append(url)
                user_data_dict['photo_url'] = url
        
        if user_data_dict not in users_data:
            users_data.append(user_data_dict)
    
    return photos_url, users_data

In [6]:
# Função para analisar fotos recuperadas

# Função para analisar fotos recuperadas

def photo_analyzer(photos_url, users_data):
    global tags, celebritie


    # Tags, descrição e reconhecimento de celebridades relacionadas a imagem

    for i in range(0, len(photos_url)):        
        celebritie, celebritie_confidence = [], []
        tags, tags_confidence = [], []

        url = photos_url[i]

        image_var = azure.describe_image(url, 1, langauge = 'en')
        celebrities_var = azure.analyze_image_by_domain('celebrities', url)
        tags_var = azure.tag_image(url)

        for c in tags_var.tags:
            tags.append(c.name)
            tags_confidence.append(c.confidence)
            
        for c in image_var.captions:
            description = c.text
            description_confidence = c.confidence
        
        for c in celebrities_var.result['celebrities']:
            celebritie.append(c['name'])
            celebritie_confidence.append(c['confidence'])
        
        for c in users_data:
            if c['photo_url'] == url:

                c['photo_tags'] = tags
                c['photo_tags_confidence'] = tags_confidence
                c['photo_descriptions'] = description
                c['photo_descriptions_confidence'] = description_confidence
                c['photo_celebrities']= celebritie
                c['photo_celebrities_confidence'] = celebritie_confidence

In [7]:
# Função que reune todos os passos para que a API funcione corretamenet

def general(theme, since, language, items):

    photos_url, tweets_data = twitter_analize(theme, since, language, items)
    photo_analyzer(photos_url, tweets_data)

    df_twitter = pd.DataFrame().from_dict(tweets_data)

    return df_twitter

In [8]:
data = general('dwayne johnson', '2021-01-01', 'en', 50)

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


In [9]:
data.loc[data['photo_url'] != '']

,id,user_name,tweet,photo_url,photo_tags,photo_tags_confidence,photo_descriptions,photo_descriptions_confidence,photo_celebrities,photo_celebrities_confidence
5,1470937857980305413,TheAnnAtomic,And then he removed his helmet and it was fuck...,http://pbs.twimg.com/media/FGnSL-uXwAYKdxk.jpg,"[sketch, drawing, line art, cartoon, illustrat...","[0.965340256690979, 0.9540235996246338, 0.8949...",a cartoon of a man and a woman,0.394727,[],[]
35,1470920543377321986,meguaud,@sucromi NAURR BFF may dwayne johnson bless ur...,http://pbs.twimg.com/tweet_video_thumb/FGnCbuB...,"[clothing, person, human face, concert, man, i...","[0.9951944947242737, 0.9948623180389404, 0.983...",Dwayne Johnson in a black shirt,0.372664,[Dwayne Johnson],[0.9998407363891602]
36,1470919710153789445,JesusHedgehog,@LiddlesTV I like the one that looks like Dway...,http://pbs.twimg.com/ext_tw_video_thumb/147091...,"[text, cartoon, smile, animation, toy, animate...","[0.9973372220993042, 0.9545409679412842, 0.909...",a person holding a toy,0.389301,[],[]
37,1470919172116951040,Vee_gmax_eevee,RT @ShinzoOshiro: Just found out that Dwayne T...,http://pbs.twimg.com/media/FGm6aqCXsAESeMs.jpg,"[furry, mammal, fur, cartoon, mascot, indoor, ...","[0.9752357006072998, 0.9383376836776733, 0.914...",a person wearing a garment,0.569091,[],[]


In [10]:
data

,id,user_name,tweet,photo_url,photo_tags,photo_tags_confidence,photo_descriptions,photo_descriptions_confidence,photo_celebrities,photo_celebrities_confidence
0,1470940001986768898,SuwakoSunday,RT @NotQuiteHereTSM: joon yorigami is dwayne t...,,,,,,,
1,1470939944084402178,vcvenus,@adamdeeks I don’t know. I rode hard for @KimC...,,,,,,,
2,1470939790392573955,medicineenjoyer,RT @NotQuiteHereTSM: joon yorigami is dwayne t...,,,,,,,
3,1470939454059778049,NotQuiteHereTSM,joon yorigami is dwayne the rock johnson's wre...,,,,,,,
4,1470939291224358915,soy_pendeho,dwayne the cock johnson,,,,,,,
5,1470937857980305413,TheAnnAtomic,And then he removed his helmet and it was fuck...,http://pbs.twimg.com/media/FGnSL-uXwAYKdxk.jpg,"[sketch, drawing, line art, cartoon, illustrat...","[0.965340256690979, 0.9540235996246338, 0.8949...",a cartoon of a man and a woman,0.394727,[],[]
6,1470937092607856643,ZachGreubel,@NetflixFilm Can't wait for Dwayne Johnson to ...,,,,,,,
7,1470936955332358149,totalsourcenews,It's the birthday of Dwayne Johnson.,,,,,,,
8,1470936499667488774,DekmarTrades,Top Speakers I love listening to when I need t...,,,,,,,
9,1470936330888790025,therealretsam,@FortniteFactsfr @ImBriskid2 are you down bad ...,,,,,,,
